<a href="https://colab.research.google.com/github/JaeyeongLee1/DSAIL_summer_internship/blob/main/FM_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [237]:
import tensorflow as tf
import sklearn
from sklearn import datasets
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler
import random

cancer = sklearn.datasets.load_breast_cancer()
scaler = MinMaxScaler()
data = scaler.fit_transform(cancer.data)
y = cancer.target
data = pd.DataFrame(data = data, columns = cancer.feature_names)

rows = data.shape[0]
n = data.shape[1]
k = 10

def g(x):
  return 1/(1+math.exp(-x))

def g_prime(x):
  return math.exp(-x)/((1+math.exp(-x))**2)

def y_hats(w0,W,V):
  y_hats = []
  for user in range(rows):
    x = data.loc[user]
    y_hat = w0+W.dot(x)
    for f in range(k):
      y_hat+=0.5*((V.T[f].dot(x))**2 - (V.T[f]**2).dot(x**2))
    y_hats.append(y_hat)
  return y_hats

def loss(w0,W,V):
  loss = 0
  y_hat = y_hats(w0,W,V)
  for user in range(rows):
    loss += -(y[user]*math.log(g(y_hat[user])+1e-15) + (1-y[user])*math.log(1-g(y_hat[user])+1e-15))
  return loss

def loss_gradient(w0,W,V):
  gradient_w0 = 0
  gradient_W = np.zeros(n)
  gradient_V = np.zeros((n,k))
  y_hat = y_hats(w0,W,V)
  for user in range(rows):
    x = data.loc[user]
    grad_w0 = 1
    grad_W = np.zeros(n)
    grad_V = np.zeros((n,k))
    for i in range(k):
      grad_W[i] = x[i]
    for f in range(k):
      precom = V.T[f].dot(x)
      for i in range(n):
        grad_V[i][f] = x[i]*precom - V[i][f]*(x[i]**2)
    if y[user] ==1:
      c = -g_prime(y_hat[user])/(g(y_hat[user]+1e-15))
      #c = -1e-3
    else:
      c = g_prime(1-y_hat[user])/((1-g(y_hat[user])+1e-15))
      #c = 1e-3
    gradient_w0 += c*grad_w0
    gradient_W += c*grad_W
    gradient_V += c*grad_V
  return (gradient_w0,gradient_W,gradient_V)
    
def GD_step(w0,W,V,alpha):
  gradient = loss_gradient(w0,W,V)
  w0 = w0 - alpha*gradient[0]
  W = W - alpha*gradient[1]
  V = V - alpha*gradient[2]
  return (w0,W,V)

def FM_fit(k,alpha,epochs):
  w0 = 0
  W = np.zeros(n)
  V = np.zeros((n,k))
  for i in range(epochs):
    w0,W,V = GD_step(w0,W,V,alpha)
  return w0,W,V

def accuracy(w0,W,V):
  predictions = [g(y_hat) for y_hat in y_hats(w0,W,V)]
  count = 0
  for i in range(rows):
    if predictions[i]>=0.5:
      if y[i] == 1:
        count += 1
    else:
      if y[i] == 0:
        count += 1
  return count/rows


In [238]:
alpha = 0.05
epochs = 100
k = 10
w0,W,V = FM_fit(k,alpha,epochs)
print("accuracy: {}".format(accuracy(w0,W,V)))
print("cross entropy loss: {}".format(loss(w0,W,V)))

accuracy: 0.9349736379613357
cross entropy loss: 86.69125125094297
